In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when

spark = SparkSession.builder.appName("CustomerOrdersJobSQL").getOrCreate()

In [3]:
customers_path = "/opt/spark-apps/input/customers.csv"
orders_path = "/opt/spark-apps/input/orders.json"
output_path_csv = "/opt/spark-apps/output/orders_enriched_csv"
output_path_parquet = "/opt/spark-apps/output/orders_enriched_parquet"

# ✅ Prod Mode (HDFS)
# customers_path = "hdfs://hdfs-namenode:9000/input/customers.csv"
# orders_path = "hdfs://hdfs-namenode:9000/input/orders.json"
# output_path_csv = "hdfs://hdfs-namenode:9000/output/orders_enriched_csv"
# output_path_parquet = "hdfs://hdfs-namenode:9000/output/orders_enriched_parquet"

In [4]:
df_customers = spark.read.option("header", True).csv(customers_path)
df_orders = spark.read.json(orders_path)

In [5]:
df_customers.createOrReplaceTempView("customers")
df_orders.createOrReplaceTempView("orders")

In [7]:
df_customers.show(), df_orders.show()

+-----------+----------+----------------+---+-------+
|customer_id|      name|           email|age|country|
+-----------+----------+----------------+---+-------+
|        101|  John Doe|john@example.com| 28|     US|
|        102|Jane Smith|jane@example.com| 34|     UK|
|        103|  Ali Khan| ali@example.com| 40|    UAE|
|        104|Rita Mehra|rita@example.com| 25|  India|
|        105| Wei Zhang| wei@example.com| 31|  China|
+-----------+----------+----------------+---+-------+

+------+-----------+--------+----------+
|amount|customer_id|order_id|    status|
+------+-----------+--------+----------+
| 250.5|        101|    5001|   shipped|
| 145.0|        102|    5002| cancelled|
|389.99|        104|    5003|processing|
| 89.99|        105|    5004|   shipped|
| 199.0|        106|    5005|  returned|
+------+-----------+--------+----------+



(None, None)

In [8]:
spark.sql("""
    SELECT *
    FROM customers
    WHERE country = 'US'
""").show()

+-----------+--------+----------------+---+-------+
|customer_id|    name|           email|age|country|
+-----------+--------+----------------+---+-------+
|        101|John Doe|john@example.com| 28|     US|
+-----------+--------+----------------+---+-------+



In [9]:
spark.sql("""
    SELECT o.order_id, c.name, o.amount,
           CASE 
               WHEN o.amount >= 200 THEN 'High Value'
               WHEN o.amount >= 100 THEN 'Medium Value'
               ELSE 'Low Value'
           END AS order_type
    FROM orders o
    JOIN customers c
      ON o.customer_id = c.customer_id
""").show()

+--------+----------+------+------------+
|order_id|      name|amount|  order_type|
+--------+----------+------+------------+
|    5001|  John Doe| 250.5|  High Value|
|    5002|Jane Smith| 145.0|Medium Value|
|    5003|Rita Mehra|389.99|  High Value|
|    5004| Wei Zhang| 89.99|   Low Value|
+--------+----------+------+------------+



In [10]:
df_enriched_op = spark.sql("""
    SELECT o.order_id, c.name, o.amount,
           CASE 
               WHEN o.amount >= 200 THEN 'High Value'
               WHEN o.amount >= 100 THEN 'Medium Value'
               ELSE 'Low Value'
           END AS order_type
    FROM orders o
    JOIN customers c
      ON o.customer_id = c.customer_id
""")

In [11]:
df_enriched_op.write.mode("overwrite").csv(output_path_csv)
df_enriched_op.write.mode("overwrite").parquet(output_path_parquet)